# 네이버 뉴스 크롤러 만들기

## Configuration

In [14]:
import requests
from bs4 import BeautifulSoup

## 사전 학습

### HTML , css

In [163]:
from IPython.core.display import HTML

html = """
<html>
    <head>
        <style>
            h3 {
                color: red;
            }
            img {
                width: 80px;
            }
            p#subtitle {
                font-size: 16px;
            }
            span.strong {
                font-weight: bold;
            }
            .description {
                font-size: 12px;
            }
            .description p {
                color: blue;
            }
        </style>
    
    </head>
        <h3>DSschool</h3>
        <p id="subtitle">만나서 반갑습니다.</p>
        
        <img src="https://bit.ly/ds-logo"/>
        
        <div class="description">
            <p>세부 정보</p>
            <ul>
                <li>장소 - <span class="strong">준경빌딩</span></li>
                <li>시간</li>
                <li>기타 - <a href="https://www.naver.com">네이버</a> </li>
            </ul>
            <ol>
                <li>장소</li>
                <li>시간</li>
                <li>기타</li>
            </ol>
        </div>
    <body>
    </body>
</html>
"""

HTML(html)

In [164]:
# html 정리
html fomater

SyntaxError: invalid syntax (<ipython-input-164-9412b4efbfa7>, line 2)

In [165]:
# 웹 페이지에서
# id 하나만 #
# class 여러개 .

In [166]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(html)

In [167]:
bs.h3.text

'DSschool'

In [168]:
bs.p.text

'만나서 반갑습니다.'

In [169]:
elements = bs.select('p')
elements

[<p id="subtitle">만나서 반갑습니다.</p>, <p>세부 정보</p>]

In [170]:
elements[0].text

'만나서 반갑습니다.'

In [171]:
elements[1].text

'세부 정보'

In [172]:
for element in elements:
    print(element.text)

만나서 반갑습니다.
세부 정보


In [186]:
bs.select('p#subtitle')

[]

In [174]:
bs.select('.description p')

[<p>세부 정보</p>]

In [175]:
elements = bs.select('.description li')

for element in elements:
    print(element.text)

장소 - 준경빌딩
시간
기타 - 네이버 
장소
시간
기타


### Request

In [47]:
import requests

url = "https://news.naver.com/main/list.nhn?listType=title"

response = requests.get(url)
response

<Response [200]>

In [52]:
bs = BeautifulSoup(response.content)
bs

<!DOCTYPE HTML>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh"/>
<meta content="width=1106" name="viewport"/>
<meta content="언론사별 속보 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="http://news.naver.com/main/list.nhn?mode=LSD&amp;mid=sec&amp;sid=" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image"/>
<meta content="언론사별 속보 뉴스를 제공합니다." property="og:description"/>
<meta content="네이버" property="og:article:author"/>
<meta content="summary" name="twitter:card"/>
<meta content="언론사별 속보 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" name="twitter:image"/>
<meta c

In [53]:
bs.title.text.strip()

'속보 홈 : 네이버 뉴스'

In [54]:
bs.h3

<h3>헤드라인 뉴스</h3>

In [59]:
bs.p

<p> <span class="newstopic_error_text">추출이 일시적으로 늦어져 최신 정보를 제공할 수 없습니다.</span> <a class="newstopic_help" href="#" onclick="news.external.OPS.helpHotTopicKeyword(); return false;" target="_blank" title="새창"><span class="newstopic_time">2019.08.31. 08:30 ~ 11:30 기준</span></a> </p>

In [60]:
bs.select('h3')

[<h3>헤드라인 뉴스</h3>, <h3> 속보</h3>, <h3>전체</h3>]

## 1. 뉴스 리스트 읽어오기

### Implementation

In [15]:
# ToDo
# 여기에 네이버 뉴스 리스트 페이지를 읽어오는 크롤러를 구현해주세요.

In [107]:
def get_news_list(html):
    bs = BeautifulSoup(html)
    elements = bs.select('#main_content .list_body ul li')

    news_list = []

    for element in elements:
        title = element.a.text
        url = element.a['href']
        company = element.span.text

        news = {
            'title' : title,
            'url' : url,
            'company' : company
        }
        news_list.append(news)

    return news_list

### Unit Tests

**Case 1-1** - 속보 페이지에 아무런 신문 기사가 없을 경우

In [108]:
html = """
<div id="main_content">
    <div class="list_body">
    </div>
</div>
"""

news_list = get_news_list(html)
assert len(news_list) == 0

**Case 1-2** - 속보 페이지에 신문기사가 한 개 있을 경우

In [109]:
html = """
<div id="main_content">
    <div class="list_body">
        <ul class="type02">
            <li>
                <a href=""></a>
                <span class="writing"></span>
            </li>
        </ul>
    </div>
</div>
"""

news_list = get_news_list(html)
assert len(news_list) == 1

**Case 1-3** - 속보 페이지에 신문기사가 두 개 있을 경우

In [110]:
html = """
<div id="main_content">
    <div class="list_body">
        <ul class="type02">
            <li>
                <a href=""></a>
                <span class="writing"></span>
            </li>
            <li>
                <a href=""></a>
                <span class="writing"></span>
            </li>
        </ul>
    </div>
</div>
"""

news_list = get_news_list(html)
assert len(news_list) == 2

**Case 1-4** - 한 개의 기사가 있는 속보 페이지에서 제목을 가져오기

In [111]:
html = """
<div id="main_content">
    <div class="list_body">
        <ul class="type02">
            <li>
                <a href="">첫 번째 신문기사</a>
                <span class="writing"></span>
            </li>
        </ul>
    </div>
</div>
"""

news_list = get_news_list(html)
assert len(news_list) == 1

first_news = news_list[0]
assert first_news['title'] == "첫 번째 신문기사"

**Case 1-5** - 한 개의 기사가 있는 속보 페이지에서 제목과 링크를 가져오기

In [112]:
html = """
<div id="main_content">
    <div class="list_body">
        <ul class="type02">
            <li>
                <a href="news1.html">첫 번째 신문기사</a>
                <span class="writing"></span>
            </li>
        </ul>
    </div>
</div>
"""

news_list = get_news_list(html)
assert len(news_list) == 1

first_news = news_list[0]
assert first_news['title'] == "첫 번째 신문기사"
assert first_news['url'] == "news1.html"

**Case 1-6** - 한 개의 기사가 있는 속보 페이지에서 제목, 링크, 신문사명을 가져오기

In [114]:
html = """
<div id="main_content">
    <div class="list_body">
        <ul class="type02">
            <li>
                <a href="news1.html">첫 번째 신문기사</a>
                <span class="writing">DS뉴스</span>
            </li>
        </ul>
    </div>
</div>
"""

news_list = get_news_list(html)
assert len(news_list) == 1

first_news = news_list[0]
assert first_news['title'] == "첫 번째 신문기사"
assert first_news['url'] == "news1.html"
assert first_news['company'] == "DS뉴스"

**Case 1-7** - 두 개의 기사가 있는 속보 페이지에서 제목, 링크, 신문사명을 가져오기

In [115]:
html = """
<div id="main_content">
    <div class="list_body">
        <ul class="type02">
            <li>
                <a href="news1.html">첫 번째 신문기사</a>
                <span class="writing">DS뉴스</span>
            </li>
            <li>
                <a href="news2.html">두 번째 신문기사</a>
                <span class="writing">TNS뉴스</span>
            </li>
        </ul>
    </div>
</div>
"""

news_list = get_news_list(html)
assert len(news_list) == 2

first_news = news_list[0]
assert first_news['title'] == "첫 번째 신문기사"
assert first_news['url'] == "news1.html"
assert first_news['company'] == "DS뉴스"

last_news = news_list[1]
assert last_news['title'] == "두 번째 신문기사"
assert last_news['url'] == "news2.html"
assert last_news['company'] == "TNS뉴스"

In [116]:
bs = BeautifulSoup(html)
elements = bs.select('#main_content .list_body ul li')

news_list = []

for element in elements:
    title = element.a.text
    url = element.a['href']
    company = element.span.text

    news = {
        'title' : title,
        'url' : url,
        'company' : company
    }
    news_list.append(news)

news_list

[{'title': '첫 번째 신문기사', 'url': 'news1.html', 'company': 'DS뉴스'},
 {'title': '두 번째 신문기사', 'url': 'news2.html', 'company': 'TNS뉴스'}]

**Case 1-8** - 실제 네이버 뉴스 속보 페이지를 크롤링하기

In [117]:
# 네이버 속보 뉴스 URL을 news_list_url 라는 이름의 변수에 할당합니다.
news_list_url = "https://news.naver.com/main/list.nhn?listType=title"

# 위 URL로 접속한다음 그 응답 결과를 가져옵니다. 이를 response라는 이름의 변수에 할당합니다.
response = requests.get(news_list_url)

# 속보 페이지에서 뉴스 리스트를 읽어옵니다.
news_list = get_news_list(response.content)

# 이 뉴스 리스트를 전부 출력합니다.
news_list

[{'title': 'MLB 투수 스캑스, 술과 진통제 복용 후 토사물에 질식사',
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=009&aid=0004419778',
  'company': '매일경제'},
 {'title': '"장고항 인근 해역서 잠수함 의심물체" 오인 신고 해프닝',
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=001&aid=0011052030',
  'company': '연합뉴스'},
 {'title': '세계 해외관광객 연간 14억명 돌파',
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=028&aid=0002466546',
  'company': '한겨레'},
 {'title': "한국당, 조국 청문회 핵심증인 채택 '양보 불가' 재확인…협상타결 난망",
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=421&aid=0004172854',
  'company': '뉴스1'},
 {'title': "가평군자원봉사센터 '사랑의 송편 만들기'",
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=003&aid=0009428668',
  'company': '뉴시스'},
 {'title': '충주 접착제 공장서 폭발·화재 …1명 실종 8명 부상',
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=437&aid=0000218696',
  'company': 'JTBC'},
 {'title': '또다시 드러난 조국의 위선, 더 이상 국민 우롱 말고 사무실의 꽃보며 자위(自慰)나 하시라 [김정재 원내대변인 논

## 2. 뉴스 상세 정보 읽어오기

### Implementation

In [ ]:
# ToDo
# 여기에 네이버 뉴스의 상세정보를 읽어오는 크롤러를 구현해주세요.

In [309]:
java

def get_news_detail_prac(html):    
    bs = BeautifulSoup(html)
    elements = bs.select('div#articleBodyContents')
    script_elements = bs.select('div#articleBodyContents Script')
    image_elements = bs.select('div#articleBodyContents .end_photo_org')
    
    if script_elements == []:
        content = elements[0].text.strip()
    else:
        content = elements[0].text.replace(script_elements[0].text, '').strip()
    
    if image_elements == []:
        image = None
    else:
        image = image_elements[0].img['src']
        
    news_detail = []
    
    news_list = {
        'content' : content,
        'image' : image
    }

    return news_list

In [ ]:
def get_news_detail(html):
    bs = BeautifulSoup(html)
    
    content_

### Unit Tests

**Case 2-1** - 뉴스 페이지에 아무런 내용이 없을 경우

In [310]:
html = """
<div id="articleBodyContents">
</div>
"""

news = get_news_detail(html)
assert news['content'] == ""

**Case 2-2** - 뉴스 페이지에 내용이 있을 경우

In [311]:
html = """
<div id="articleBodyContents">
    월드컵 응원하는 서울시민들
</div>
"""

news = get_news_detail(html)
assert news['content'] == "월드컵 응원하는 서울시민들"

**Case 2-3** - 뉴스 페이지에 내용이 있으나, 크롤링할 필요 없는 자바스크립트 코드가 있을 경우

In [312]:
html = """
<div id="articleBodyContents">
    <script type="text/javascript">
        // flash 오류를 우회하기 위한 함수 추가
        function _flash_removeCallback() {}
    </script>

    월드컵 응원하는 서울시민들
</div>
"""

news = get_news_detail(html)
assert news['content'] == "월드컵 응원하는 서울시민들"

**Case 2-4** - 뉴스 페이지에 사진이 있는 경우

In [313]:
html = """
<div id="articleBodyContents">
    <script type="text/javascript">
        // flash 오류를 우회하기 위한 함수 추가
        function _flash_removeCallback() {}
    </script>
    
    <span class="end_photo_org">
        <img src="image1.png" />
    </span>

    월드컵 응원하는 서울시민들
</div>
"""

news = get_news_detail(html)

assert news['content'] == "월드컵 응원하는 서울시민들"
assert news['image'] == "image1.png"


**Case 2-5** - 실제 네이버 뉴스 상세 페이지를 크롤링하기

In [314]:
# 네이버 뉴스 URL을 news_url 라는 이름의 변수에 할당합니다.
news_url = "https://news.naver.com/main/read.nhn?oid=091&aid=0007323612"

# 위 URL로 접속한다음 그 응답 결과를 가져옵니다. 이를 response라는 이름의 변수에 할당합니다.
response = requests.get(news_url)

# 위 페이지에서 뉴스 정보를 가져옵니다. 이를 news라는 이름의 변수에 할당합니다.
news = get_news_detail(response.content)

# 이 news 변수에 할당된 값을 출력합니다.
news

{'content': '119th US Open ChampionshipTiger Woods of the US hits his tee shot on the second hole during the third round of the 119th US Open Championship at the Pebble Beach Golf Links in Pebble Beach, California, USA, 15 June 2019. The tournament is being played from 13 June to 16 June.  EPA/ETIENNE LAURENT▶네이버 홈에서 [연합뉴스] 채널 구독하기▶뭐 하고 놀까? #흥  ▶쇼미더뉴스! 오늘 많이 본 뉴스영상',
 'image': 'https://imgnews.pstatic.net/image/091/2019/06/16/PEP20190616045001848_P2_20190616042121869.jpg?type=w647'}

### 3. 전체 페이지 읽어오기

In [315]:
# 네이버 속보 뉴스 URL을 news_list_url 라는 이름의 변수에 할당합니다.
news_list_url = "https://news.naver.com/main/list.nhn?listType=title"

# 위 URL로 접속한다음 그 응답 결과를 가져옵니다. 이를 response라는 이름의 변수에 할당합니다.
response = requests.get(news_list_url)

# 속보 페이지에서 뉴스 리스트를 읽어옵니다. 이 결과를 news_summary_list 라는 이름의 변수에 할당합니다.
news_summary_list = get_news_list(response.content)

# 최종적으로 뉴스 정보를 전부 저장할 news_list 라는 이름의 변수를 만듭니다. 여기에는 리스트가 들어갑니다.
news_list = []

# news_summary_list 안에 있는 모든 뉴스 정보를 반복문(for 문)으로 읽어옵니다.
for news_summary in news_summary_list:
    # 이 뉴스 정보의 제목(title)을 읽어옵니다. 이를 title이라는 이름의 변수에 할당합니다.
    title = news_summary['title']

    # 이 뉴스 정보의 주소(URL)을 읽어옵니다. 이를 content_url라는 이름의 변수에 할당합니다.
    content_url = news_summary['url']
    
    # 진행 상황을 확인하기 위해 로그를 출력합니다.
    print(f"Crawling {title}...")

    # 뉴스 정보의 주소로 접속한 뒤 그 응답 결과를 가져옵니다. 이를 response라는 이름의 변수에 할당합니다.
    response = requests.get(content_url)
    
    # 위 응답 결과에서 뉴스의 상세 정보를 크롤링합니다.
    news_detail = get_news_detail(response.content)
    
    # 여태까지 크롤링한 모든 정보(제목, 주소, 내용 등)를 전부 조합하여 news라는 이름의 변수에 할당합니다.
    news = {
        'title': news_summary['title'],
        'url': news_summary['url'],
        'company': news_summary['company'],
        'content': news_detail['content'],
        'image': news_detail['image'],
    }
    
    # 위 뉴스 정보를 news_list라는 리스트 안에 추가합니다.
    news_list.append(news)
    
# news_list 변수 안에 들어있는 모든 뉴스 정보를 출력합니다.
news_list

Crawling epaselect USA TENNIS US OPEN GRAND SLAM 2019...
Crawling <포토> 대화하는 나경원-김문수...
Crawling 일본 고속열차 승객 큰 짐 사전예약제 도입…도쿄올림픽 테러 대책...
Crawling 日 성인용 기저귀 시장 꾸준히 성장…액티브 시니어 사용 증가 덕분...
Crawling 결혼 후 첫째아 출산하기까지 평균 2.16년...
Crawling 주한미군, 용산기지 편의시설 10월 안에 대부분 폐쇄...
Crawling '다문화 행사' 국민의례...
Crawling epaselect USA BASEBALL MLB...
Crawling <포토> 자유한국당 장외투쟁서 만난 김진태-김문수...
Crawling [달콤한 사이언스] 질풍노도 ‘중2병’ 우리아이들 머릿 속을 보니…...
Crawling ‘6만원’ 훔친 죄로 36년간 감옥살이 한 美남성 사연...
Crawling 일본 컬링 선수 "한국 딸기 맛있어요"  말했다가…...
Crawling [실시간 수도권 날씨] 오후 3시 현재 대체로 맑음...
Crawling [실시간 전국 날씨] 오후 3시 현재 대체로 흐림...
Crawling [부고] 차영구(한국기원 홍보미디어팀장)씨 부친상...
Crawling 식약처, “엘러간 유방 보형물 9월까지 보상방만 마련”...
Crawling 韓, APEC 고위관리회의서 日 수출규제에 "깊은 유감" 표명...
Crawling 삼양목장에서 보내는 휴일...
Crawling [의왕소식] 아동·여성안전지역연대 운영위 개최 등...
Crawling 당원들에게 인사하는 나경원 원내대표...
Crawling 나경원 '文정권 규탄'...
Crawling 규탄 발언하는 나경원 원내대표...
Crawling <포토> 국민의례하는 황교안-나경원...
Crawling 오신환 “고위 공직자들 자제 입시비리 조사해야”… 경북대 총학 요구에 화답...
Crawling 나경원 "조국 가족, 어떻게든 불러 진짜 청문회 하겠다"...
Crawl

[{'title': 'epaselect USA TENNIS US OPEN GRAND SLAM 2019',
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=091&aid=0007459932',
  'company': 'EPA연합뉴스',
  'content': 'US Open 2019 tennis tournament in New Yorkepaselect epa07805837 Novak Djokovic of Serbia in action against Denis Kudla of the USA during the fifth day of the US Open Tennis Championships at the USTA National Tennis Center in Flushing Meadows, New York, USA, 30 August 2019. The US Open runs from 26 August through to 08 September 2019.  EPA/JASON SZENES▶확 달라진 연합뉴스 웹을 만나보세요▶네이버 [연합뉴스] 채널 구독   ▶뭐 하고 놀까? #흥',
  'image': 'https://imgnews.pstatic.net/image/091/2019/08/31/PEP20190831121701848_P2_20190831150227848.jpg?type=w647'},
 {'title': '<포토> 대화하는 나경원-김문수',
  'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=119&aid=0002349456',
  'company': '데일리안',
  'content': '자유한국당 나경원 원내대표와 김문수 전 경기도지사가 31일 오후 서울 종로구 경복궁역 인근에서 열린 자유한국당 문재인 정권 규탄 집회에서 대화를 하고 있다.ⓒ데일리안 홍금표 기자데일리안 홍금표 기자 (goldpyo@dailian.co

### 4. CSV로 저장하기

In [ ]:
# 데이터 분석 패키지 판다스(Pandas)를 가져옵니다.
# 이를 줄여서 pd라는 축약어로 사용합니다.
import pandas as pd

# 앞서 읽어온 뉴스 정보(news_list)를 판다스의 데이터프레임 형태로 변환합니다.
news_list = pd.DataFrame(news_list)

# 이 결과를 화면에 출력합니다.
news_list

In [ ]:
# 이 뉴스 정보를 CSV라는 형식으로 저장합니다.
news_list.to_csv("news.csv")